## TensorFlow 模型的保存和载入
需要注意的细节：  
1、由于后续需要去除模型中具体的数据，所以在定义网络是需要给tensor和variable取名字。  
### 使用 saver 来保存模型


In [ ]:
saver = tf.train.Saver()  
with tf.Session() as sess:  
   sess.run()  
   saver.save(sess, 'models/my_model.ckpt')

### 模型的载入

"models/my_model.ckpt.meta"  文件中保存的是模型的网络结构  
"models/my_model.ckpt" 文件中保存的是模型的参数

In [ ]:
with tf.Session() as sess:
    # 载入模型结构
    saver = tf.train.import_meta_graph('models/my_model.ckpt.meta')
    # 载入模型参数（保存的是参数的数值）
    saver.restore(sess, 'models/my_model.ckpt')
    
    # 根据 tensor 的名字获取到对应的 tensor
    # 之前保存模型的时候模型输出保存为 output, "0" 是保存模型参数时自动加上的，所以这里也要加上
    output = sess.graph.get_tensor_by_name('output:0')
    accuracy = sess.graph.get_tensor_by_name('accuracy:0')
    train_step = sess.graph.get_operation_by_name('train')   # 获取操作
    
    # 把测试集喂到网络中计算准确率
    # x-input 是模型数据的输入，":0" 是保存模型参数时自动加上的，所以这里也要写上
    # y-input 是模型的标签输入，":0" 是保存模型参数时自动加上的，所以这里也要写上
    print(sess.run(accuracy, feed_dict = {'x-input:0': minst.test.images, 
                                          'y-input:0': minst.test.labels}))
    
    # 在原来模型的基础上再训练 11 个周期
    for epoch in range(11):
        for batch in range(n_batch):
            batch_xs, batch_ys = minst.train.next_batch(batch_size)
            sess.run(train_step, feed_dict = {'x-input:0': batch_xs,
                                              'y-input:0': batch_ys})
        # 计算模型的准确率
        acc = sess.run(accuracy, feed_dict = {'x-input:0': minst.test.images, 
                                              'y-input:0': minst.test.labels})
        print(acc)

### 模型的载入（2）
只有模型参数的文件，没有网络结构的文件。此时需要自己先定义网络结构，然后再载入模型的参数